# Code

## 0 데이터 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import Trainer, TrainingArguments
from transformers import get_scheduler
from transformers import EarlyStoppingCallback

import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
train=pd.read_csv('./train.csv',encoding = 'utf-8-sig')
test=pd.read_csv('./test.csv',encoding = 'utf-8-sig')

In [ ]:
# 체크포인트 경로 설정
checkpoint_dir = "./체크포인트 경로"

# 최신 체크포인트 찾기 (있으면 로드, 없으면 None)
checkpoint_dirs = glob.glob(os.path.join(checkpoint_dir, "checkpoint-*"))
latest_checkpoint = sorted(checkpoint_dirs, key=lambda x: int(x.split("-")[-1]))[-1] if checkpoint_dirs else None

Found checkpoint: /content/drive/MyDrive/Euron-project/ljy-checkpoint/checkpoint-957


In [ ]:
if latest_checkpoint:
    print(f"Loading checkpoint from: {latest_checkpoint}")
    # 저장된 체크포인트에서 모델 로드
    model = T5ForConditionalGeneration.from_pretrained(latest_checkpoint)
else:
    print("No checkpoint found. Start from scratch.")
    model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")

tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

Loading checkpoint from: /content/drive/MyDrive/Euron-project/ljy-checkpoint/checkpoint-957


입력 토큰 개수 제한 딱히 없음

In [ ]:
# model.config.use_cache = False

In [ ]:
train.head()

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...


In [ ]:
test.head()

,ID,input
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜...
1,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰..."
2,TEST_0002,쥔차 붉찐졀행욘. 삶먼섶 멂묽럿턴 혹텔 중웨 쬐약위였습뉜따. 칙어뉜쥐 샤쨩윈쥐 쩨끄...
3,TEST_0003,붊 맛짚~~ 글련뎨 방움잃 뮈흙퍄녜용. 충칸 쏘움광 팔쿄닛갸 잊중짱임 야뉘럇셧 팜몌...
4,TEST_0004,빻 샹택는 쥔쨔 폐헐 칙젓뉜테 쩐맣은 죠하욧. 뽀읾럭카 알쥬 찬쟌합꿰 똘앝썬 츄어서...


## 2 ET5 사용

In [ ]:
# train / validation 분리
train_df, val_df = train_test_split(train, test_size=0.15, random_state=42)

In [ ]:
train_df.head()

,ID,input,output
5527,TRAIN_05527,"쟘만 짧럭교 옌악헷떤 커란 쩨쿠인닐 눋엇는덴, 뮨 얼꼬 틂언깐는 쑨깐 먈무뉘 맡컷썩...","잠만 자려고 예약했던 거라 체크인이 늦었는데, 문 열고 들어가는 순간 말문이 막혔어..."
388,TRAIN_00388,삽임툴돛 넒코 퀘쩍깎꼬 앍추 좋앓오.,사이트도 넓고 쾌적하고 아주 좋아요.
836,TRAIN_00836,쬐교의 멍쇽잎닢탔^^ 꼭 한뻔 왑뽈 만한 쑥쇼 삑!,최고의 명소입니다^^ 꼭 한번 와볼 만한 숙소 픽!
3948,TRAIN_03948,추찻창독 널교 끈쩌엔 며긁 곧또 만코 욧션부 쭝 쬘공잎닝댜.,주차장도 넓고 근처에 먹을 곳도 많고 오션뷰 중 최고입니다.
6843,TRAIN_06843,켭삐 맛있코 쑥쑈 좋앝숲닙댜!,커피 맛있고 숙소 좋았습니다!


In [ ]:
val_df.head()

,ID,input,output
6927,TRAIN_06927,온션퓨 갸셩피 쑥쏘웩오! 별섶 듀 펀쳅 빵물닒예옷! 힐륑햐려 왔눈뎨 완젼 힐륑하꾜 ...,오션뷰 가성비 숙소에요! 벌써 두 번째 방문이에요! 힐링하러 왔는데 완전 힐링하고 ...
1669,TRAIN_01669,밧댜예셔 갖캅귄 한떼 숲 태뮨엔 젼허 빳타갔 뽀이쥐 않눈 굣윕님타. 칵깝임 잊여됴 ...,바다에서 가깝긴 한데 숲 때문에 전혀 바다가 보이지 않는 곳입니다. 가까이 있어도 ...
9499,TRAIN_09499,찜꿀루웨 떡폭뀌윈쮜 낌찧칟캐윈짇 국묾위 묻뗘있곱 찜쿠루 청숀눈 않옜 얀 한뉨 봐오....,침구류에 떡볶이인지 김치찌개인지 국물이 묻어있고 침구류 청소는 아예 안 하니 봐요....
3287,TRAIN_03287,"줍번예 룟뒈빼꽈쩜밑 잇교, 욺씩 싸왈섦 편힙 잚 놂타 왓엶오. 좋은 횹뗄른 앉닉곬,...","주변에 롯데백화점이 있고, 음식 사와서 편히 잘 놀다 왔어요. 좋은 호텔은 아니고,..."
2973,TRAIN_02973,쀼룝 핥 잃 딴 함!! 룸 켠띠션 췌샹!! 굴렇냐 옐뤼뻬인떡갔 악깐 훈들려 풀않청한...,뷰로 할 일 다 함!! 룸 컨디션 최상!! 그러나 엘리베이터가 약간 흔들려 불안정한...


In [ ]:
test.head()

,ID,input
0,TEST_0000,녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜...
1,TEST_0001,"풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰..."
2,TEST_0002,쥔차 붉찐졀행욘. 삶먼섶 멂묽럿턴 혹텔 중웨 쬐약위였습뉜따. 칙어뉜쥐 샤쨩윈쥐 쩨끄...
3,TEST_0003,붊 맛짚~~ 글련뎨 방움잃 뮈흙퍄녜용. 충칸 쏘움광 팔쿄닛갸 잊중짱임 야뉘럇셧 팜몌...
4,TEST_0004,빻 샹택는 쥔쨔 폐헐 칙젓뉜테 쩐맣은 죠하욧. 뽀읾럭카 알쥬 찬쟌합꿰 똘앝썬 츄어서...


In [ ]:
train_input_lens = [len(tokenizer.encode(text, add_special_tokens=True)) for text in train_df["input"].tolist()]
train_output_lens = [len(tokenizer.encode(text, add_special_tokens=True)) for text in train_df["output"].tolist()]
val_input_lens = [len(tokenizer.encode(text, add_special_tokens=True)) for text in val_df["input"].tolist()]
val_output_lens = [len(tokenizer.encode(text, add_special_tokens=True)) for text in val_df["output"].tolist()]

print(f"Train Input 최대 길이: {max(train_input_lens)}")
print(f"Train Output 최대 길이: {max(train_output_lens)}")
print(f"Validation Input 최대 길이: {max(val_input_lens)}")
print(f"Validation Output 최대 길이: {max(val_output_lens)}")

print(f"Train Input 평균 길이: {np.mean(train_input_lens):.2f}")
print(f"Train Output 평균 길이: {np.mean(train_output_lens):.2f}")

Train Input 최대 길이: 1449
Train Output 최대 길이: 605
Validation Input 최대 길이: 1423
Validation Output 최대 길이: 659
Train Input 평균 길이: 97.54
Train Output 평균 길이: 42.45


In [ ]:
a1 = int(np.percentile(train_input_lens, 99.5))
a2 = int(np.percentile(train_output_lens, 99.5))  # 상위 99% 길이
max_input_length = 512
max_output_length = 256

print(f"99% : {a1}, {a2}")
print(f"설정할 max_length: 입력 {max_input_length}, 출력 {max_output_length}")

99% : 570, 249
설정할 max_length: 입력 512, 출력 256


In [ ]:
# 학습 데이터 샘플 확인
print("Train input sample:", train_df["input"].iloc[0])
print("Train output sample:", train_df["output"].iloc[0])

# 토크나이징된 샘플 확인
print("Encoded Train Input:", tokenizer.encode(train_df["input"].iloc[0], add_special_tokens=True))

Train input sample: 쟘만 짧럭교 옌악헷떤 커란 쩨쿠인닐 눋엇는덴, 뮨 얼꼬 틂언깐는 쑨깐 먈무뉘 맡컷썩오. 쩡맑 뿌.. 쬐꾜잎뉘답 ㅋㅋ 짱툴 야닒 쟉뿜 깟탓어욤. 읽출 볼렴코 꺼툰울 껏코 잦눈뎨, 했빛위 한 뭍떠키 솟야쥔눈 계 헌씰갊 업얻곯 운슬읽 짬 얘푸터락곰오. 친찻 꽝얀덱쿄 퓰료는 췌곪.. 얀찜 죠쉭콰 웰켬 꾹낑, 뜰륑클톱 좋앗숩니타. 췸균와 빎찧묾뿜돌 깔쿰했꼬 천반적욺롬 넒묻 좋았썩욤. 꽝얀릿 받댜 뉴늙론 많휙 닮악카오.
Train output sample: 잠만 자려고 예약했던 거라 체크인이 늦었는데, 문 열고 들어가는 순간 말문이 막혔어요. 정말 뷰.. 최고입니다 ㅋㅋ 창틀 안이 작품 같았어요. 일출 보려고 커튼을 걷고 잤는데, 햇빛이 한 무더기 쏟아지는 게 현실감 없었고 윤슬이 참 예쁘더라고요. 진짜 광안대교 뷰로는 최고.. 아침 조식과 웰컴 쿠키, 드링크도 좋았습니다. 침구와 비치물품도 깔끔했고 전반적으로 너무 좋았어요. 광안리 바다 눈으로 많이 담아가요.
Encoded Train Input: [9186, 9166, 9089, 9082, 9224, 18671, 17428, 9721, 9186, 19282, 10350, 44236, 31847, 11008, 9576, 9186, 44775, 10365, 9225, 16450, 9186, 9165, 9066, 9069, 15028, 9197, 13734, 9188, 9186, 44988, 13266, 12787, 9186, 9167, 9069, 9060, 10465, 19635, 9197, 9186, 43737, 19635, 9186, 9165, 9098, 9066, 9511, 30940, 11148, 17471, 20662, 9429, 9187, 9186, 44032, 23812, 19398, 9187, 9187, 9186, 43659, 9164, 9120, 9086, 20592, 30940, 12645, 9417, 16488, 21181, 10299, 9165, 9069, 9076

In [ ]:
train_encodings = tokenizer(
    train_df["input"].tolist(),
    max_length=max_input_length,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
train_labels_encodings = tokenizer(
    train_df["output"].tolist(),
    max_length=max_output_length,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

val_encodings = tokenizer(
    val_df["input"].tolist(),
    max_length=max_input_length,
    padding=True,
    truncation=True,
    add_special_tokens=True
)
val_labels_encodings = tokenizer(
    val_df["output"].tolist(),
    max_length=max_output_length,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
test_input_lens = [len(tokenizer.encode(text, add_special_tokens=True)) for text in test["input"].tolist()]
print(max(test_input_lens))

1996


In [ ]:
test_encodings = tokenizer(
    test["input"].tolist(), max_length=1996, padding=True, truncation=True, add_special_tokens=True
)

In [ ]:
class SpellCorrectionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels_encodings=None):
        self.encodings = encodings
        self.labels_encodings = labels_encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels_encodings is not None:
            labels = self.labels_encodings["input_ids"][idx]
            labels = [l if l != tokenizer.pad_token_id else -100 for l in labels]  # 패딩을 -100으로 변경
            item["labels"] = torch.tensor(labels)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
# 데이터셋 정의
train_dataset = SpellCorrectionDataset(train_encodings, train_labels_encodings)
val_dataset = SpellCorrectionDataset(val_encodings, val_labels_encodings)

In [ ]:
# 정답 레이블 없이 데이터셋 생성
test_dataset = SpellCorrectionDataset(test_encodings)

Trainer 추가 아이디어:

- learning rate scheduler 추가 (linear warmup + decay)
- eval_steps 추가하여 더 자주 평가
- early stopping

\+ learning_rate: ET5의 경우 3e-5 ~ 1e-4 사이에서 최적값을 찾는 것이 일반적.

## Train

In [ ]:
# Trainer 설정
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    save_strategy="steps",
    save_steps=max(len(train_dataset) // 10, 500),
    save_total_limit=1, # 체크포인트 개수 제한
    eval_strategy="steps",
    eval_steps=max(len(train_dataset) // 10, 500),
    load_best_model_at_end=True, # 가장 좋은 모델 로드
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    fp16=True, # 메모리 부족할 때 (mixed precision 활성화)
    gradient_checkpointing=True, # OOM 에러 발생 X -> False 가능
    logging_dir="./로그 저장 경로", # 로그 저장 위치 (TensorBoard 연동 가능)
    logging_steps=100, # 학습 로그 출력 빈도
    learning_rate=5e-5,
    warmup_ratio=0.1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
print(os.path.exists(training_args.output_dir))

True


In [ ]:
# 체크포인트 경로 확인 후 학습 시작
trainer.train(resume_from_checkpoint=latest_checkpoint if latest_checkpoint else None)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sn031004 (sn03) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
957,0.567500,0.534181


In [ ]:
# %load_ext tensorboard

# # TensorBoard 실행 (logs 폴더를 모니터링)
# %tensorboard --logdir ./logs

## Inference

In [ ]:
# 프롬프트 정의
prompt = "난독화된 한글을 올바르게 복원해 주세요. 자연스럽고 올바른 한국어 문장을 생성하는 것이 가장 중요합니다. 참고로, 일부 단어에 쌍자음이 포함되어 있을 경우, 올바른 한국어가 아니라면 기본자음으로 변환하면 자연스러운 문장이 될 수도 있습니다."

# 입력 텍스트에 프롬프트 추가
def add_prompt_to_input(text):
    return f"{prompt}\n\n입력: {text}\n출력:"

# 테스트 데이터에 프롬프트 추가
test_inputs_with_prompt = [add_prompt_to_input(text) for text in test["input"].tolist()]

# 프롬프트가 추가된 입력으로 새로운 인코딩 생성
test_encodings = tokenizer(
    test_inputs_with_prompt,
    max_length=2055,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 데이터셋 재생성
test_dataset = SpellCorrectionDataset(test_encodings)

In [ ]:
# 기존 submission 파일을 불러와서 진행할 수도 있음
submission_path = "./결과 저장 파일 이름.csv"

try:
    submission = pd.read_csv(submission_path, encoding='utf-8-sig')
    existing_outputs = submission["output"].tolist()
except FileNotFoundError:
    submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')
    existing_outputs = []

In [ ]:
# 테스트 데이터셋을 어디까지 처리했는지 확인
start_idx = len(existing_outputs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

model.eval()
with torch.no_grad():
    for i, batch in tqdm(enumerate(test_dataloader, start=start_idx), total=len(test_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model.generate(
            **batch,
            max_length=1970,
            num_beams=3,
            length_penalty=1.0,
            early_stopping=True
        )

        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 결과를 바로 CSV에 저장
        submission.loc[i, "output"] = decoded_output
        submission.to_csv(submission_path, index=False, encoding='utf-8-sig')

  0%|          | 0/212 [00:00<?, ?it/s]